In [ ]:
import pandas as pd
import numpy as np


song_data = pd.read_csv('../input/19000-spotify-songs/song_data.csv') 
song_info = pd.read_csv('../input/19000-spotify-songs/song_info.csv') 


In [ ]:
song_data.head()

In [ ]:
song_info.head()

In [ ]:
song_info[song_info['song_name'] == 'Footloose']

# Convert all str to lower case

In [ ]:
song_info['playlist'] = song_info['playlist'].str.lower()
song_info['song_name'] = song_info['song_name'].str.lower()
song_info['artist_name'] = song_info['artist_name'].str.lower()
song_info['album_names'] = song_info['album_names'].str.lower()

song_data['song_name'] = song_data['song_name'].str.lower()



#song_info[song_info['playlist'].str.contains('pop')]


# Merging the two datasets into one

In [ ]:
song_df = song_data.copy()
song_df['artist_name'] = song_info['artist_name']


song_df = song_df.drop_duplicates(subset=['song_name','artist_name'])

song_info = song_info.drop_duplicates(subset=['song_name','artist_name'])


song_df = song_df.reset_index()
song_df = song_df.drop('index',axis=1)

song_df.head()

# check if the numeric features follow Gaussian distribution

In [ ]:
from scipy.stats import shapiro

stat,p =shapiro(song_df[['song_popularity', 'acousticness',
       'danceability', 'energy', 'instrumentalness', 'key', 'liveness',
       'loudness', 'audio_mode', 'speechiness', 'tempo','audio_valence']])

print("stat: ",stat,":","p :",p)
if p>0.05:
    print("gaussian distribution - fail to reject H0")
else:
    print("not gaussian distribution - reject H0")

In [ ]:
song_df = song_df.drop(['time_signature','song_duration_ms'], axis=1)
song_df.head()

# Standardise using MinMaxScaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler


minmaxscaler = MinMaxScaler()
minmaxscaled = minmaxscaler.fit_transform(song_df[['song_popularity', 'acousticness',
       'danceability', 'energy', 'instrumentalness', 'key', 'liveness',
       'loudness', 'audio_mode', 'speechiness', 'tempo','audio_valence']])
songs_normalized = pd.DataFrame(minmaxscaled, columns=[['song_popularity', 'acousticness',
       'danceability', 'energy', 'instrumentalness', 'key', 'liveness',
       'loudness', 'audio_mode', 'speechiness', 'tempo','audio_valence']])

In [ ]:
song_df_normalized = song_df.copy()
song_df_normalized.head()



In [ ]:

song_df_normalized[['song_popularity', 'acousticness',
       'danceability', 'energy', 'instrumentalness', 'key', 'liveness',
       'loudness', 'audio_mode', 'speechiness', 'tempo','audio_valence']] = songs_normalized
#song_df_normalized.drop(['bpm','nrgy','dnce','dB','live','val','acous','spch','pop'],axis=1,inplace=True)
song_df_normalized.head()

# create feature matrix

In [ ]:
song_features=song_df_normalized.set_index("song_name")
song_features.drop(["artist_name"],axis=1,inplace=True)
song_features.head()

In [ ]:
song_features.info()

In [ ]:
song_features.tail()

# Use cosine similarity based NN algo to calculate min distances and find similar songs

In [ ]:
from scipy.sparse import csr_matrix

song_features_csr = csr_matrix(song_features.values)

In [ ]:
song_features_csr 

In [ ]:
song_features.values

In [ ]:
from sklearn.neighbors import NearestNeighbors

model_nn = NearestNeighbors(metric='cosine',algorithm='brute')
model_nn.fit(song_features_csr)

In [ ]:
temp = song_features.copy()
temp.reset_index(inplace=True)
songsearch = 'footloose'
songsearch = songsearch.lower()
song_index = temp.index[temp['song_name'] ==songsearch].tolist()[0]
print(song_index)

print(song_features.index[song_index])


distances,indices = model_nn.kneighbors(X = song_features.iloc[song_index,:].values.reshape(1,-1), n_neighbors=6)

for i in range(0, len(distances.flatten())):
    if i == 0:
        print("Recommendation for ",song_features.index[song_index],"are: ")
    else:
        print(i,": ",song_features.index[indices.flatten()[i]], "| distance= ",distances.flatten()[i])

# find artist of song

In [ ]:
songgname = 'footloose'
songgname = songgname.lower()
songg = song_info[song_info['song_name'] == songgname]
#songg['artist_name'].unique().tolist()[0]
songg

# find album of song

In [ ]:
songgname = 'red red wine'
songgname = songgname.lower()
songg = song_info[song_info['song_name'] == songgname]
#songg['album_names'].unique().tolist()
songg

# find songs of artist 

In [ ]:
artistname = 'aretha franklin'
artistname = artistname.lower()
artist = song_df[song_df['artist_name'] == artistname]
artist['song_name'].unique().tolist()
#artist

# splitting main df as per playlists 

In [ ]:
song = 'i say a little prayer'
song = song.lower()

In [ ]:
playlist = song_info[song_info['song_name'] == song]
x = playlist['playlist'].unique().tolist()
x[0]

In [ ]:
temp_df = song_df.copy()
temp_df[['artist_name','playlist']] = song_info[['artist_name','playlist']]
temp_df = temp_df.drop_duplicates(subset=['song_name','artist_name'])


temp_df = temp_df.reset_index()
temp_df = temp_df.drop('index',axis=1)

temp_df[temp_df['playlist'] == x[0]]